In [1]:
import torch
import pickle
dir_safety = '/home/artur/Schreibtisch/DiscriminatingHyperplaneFork/discriminating-hyperplane/Double_Learning_Safety_Performance/data/double_learner/double_learner-customEnvs/CartPole_s43/terminated/failed_model_6999_7449.pt'
dir_actor = "/home/artur/Schreibtisch/DiscriminatingHyperplaneFork/discriminating-hyperplane/Double_Learning_Safety_Performance/data/double_learner/double_learner-customEnvs/CartPole_s43/terminated/failed_ac_6999_7449.pt"
dir_traj = "/home/artur/Schreibtisch/DiscriminatingHyperplaneFork/discriminating-hyperplane/Double_Learning_Safety_Performance/data/double_learner/double_learner-customEnvs/CartPole_s43/terminated/replay_buffer_samples:6999_7449.pkl"


safety_actor = torch.load(dir_safety)
preformance_actor = torch.load(dir_actor)
with open(dir_traj, "rb") as f:
    loaded_trajectory = pickle.load(f)

/tmp/ipykernel_103609/1641651120.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  safety_actor = torch.load(dir_safety)
/tmp/ipykernel_103609/1641651120.py:9: FutureWarni

In [2]:
import gymnasium
from gymnasium.envs.registration import register
from envs.cartpole_pret import CartPole
register(
    id="customEnvs/CartPole",
    entry_point="envs.cartpole_pret:CartPole",
)
def make_env(env_id):
    def thunk():
        env = gymnasium.make(env_id, render_mode="human",focus = 1.6, rewardtype = 1, debug_hyperplanes_render=True)
        env = gymnasium.wrappers.RecordEpisodeStatistics(env)
        return env

    return thunk
env_fn = make_env("customEnvs/CartPole")
env = env_fn()

/home/artur/miniconda3/envs/fsrl/lib/python3.8/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [ ]:

import numpy as np
for index in range(2,100):
    starting_state = list(loaded_trajectory.observations[index].detach().cpu().numpy())
    next_desired_action = loaded_trajectory.actions[index].detach().cpu().numpy()
    a, a_h, b_h, v = safety_actor.stepEval(torch.as_tensor(starting_state, dtype=torch.float32))
    filtered_action, filtered , projected= safety_actor.filter_actions(a_h,b_h,next_desired_action)
    threshold = np.clip((b_h/a_h)[0],-1,1)
    if(a_h>0):
        to_right_is_dangerous = False
    else:
        to_right_is_dangerous = True
    if filtered:
        safety_actor.filter_actions(a_h,b_h,next_desired_action)
        print(filtered_action)
        print(threshold)
        print(to_right_is_dangerous)
        print(next_desired_action)
    env.set_values_for_debug_render(filtered_action,threshold,to_right_is_dangerous, next_desired_action)
    o, info = env.reset(options={"state":starting_state})
    input("Press Enter to continue...")



/home/artur/Schreibtisch/DiscriminatingHyperplaneFork/discriminating-hyperplane/Double_Learning_Safety_Performance/core.py:392: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  a = torch.tensor([performance_action])
